# Реализация частеречной разметки (POS tagging) посредством нейронных сетей, основное задание

С точки зрения глубокого обучения, частеречная разметка - задача предсказания последовательности вывода выровненная с последовательностью ввода. 
<img src=https://i.stack.imgur.com/6pdIT.png width=320>

Существуют несколько проблем, описываемых этой задачей:
- Частеречная разметка - сопутствующая задача для большого числа проблем обработки естественного языка;
- Извлечение именованных сущностей - для чатботов и поисковых роботов;
- Предсказание структур белка - для биоинформатики.

## Загрузка данных

In [1]:
# Импорт необходимых библиотек
import nltk                   # Библиотека для работы с естественным языком
import sys                    # Библиотека для доступа к системным параметрам и функциям
import numpy as np            # Библиотека для реализации функций линейной алгебры

# Загрузка корпуса Brown - коллекции документов на английском, созданной в 1961 году
# Особенность этого корпуса в том, что документы там уже токенизированы и размечены
nltk.download('brown')
# Загрузка альтернативного набора меток, обозначающих части речи
nltk.download('universal_tagset')

# Запись корпуса Brown в переменную, вместе с альтернативной частеречной разметкой
data = nltk.corpus.brown.tagged_sents(tagset='universal')
# Ограничение списка разметки
all_tags = ['#EOS#','#UNK#','ADV', 'NOUN', 'ADP', 'PRON', 'DET', '.', 'PRT', 'VERB', 'X', 'NUM', 'CONJ', 'ADJ']

# Приведение слов в корпусе к нижнему регистру, определение каждого предложения в свой список
data = np.array([ [(word.lower(),tag) for word, tag in sentence] for sentence in data ])

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [2]:
# Визуальное исследование элемента набора данных data
data[10]

[('it', 'PRON'),
 ('urged', 'VERB'),
 ('that', 'ADP'),
 ('the', 'DET'),
 ('city', 'NOUN'),
 ('``', '.'),
 ('take', 'VERB'),
 ('steps', 'NOUN'),
 ('to', 'PRT'),
 ('remedy', 'VERB'),
 ("''", '.'),
 ('this', 'DET'),
 ('problem', 'NOUN'),
 ('.', '.')]

In [3]:
# Импорт библиотеки для разделения данных на обучающие и проверочные
from sklearn.model_selection import train_test_split
# Импорт необходимых библиотек для отображения элементов набора данных
from IPython.display import HTML, display

In [4]:
# Разделение данных на обучающие (75%) и проверочные (25%)
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

In [5]:
def draw(sentence):
  """Функция для удобочитаемого отображения загруженного частеречно размеченного текста"""
  words, tags = zip(*sentence)
  display(HTML('<table><tr>{tags}</tr>{words}<tr></table>'.format(
                words = '<td>{}</td>'.format('</td><td>'.join(words)),
                tags = '<td>{}</td>'.format('</td><td>'.join(tags)))))

In [6]:
# Визуальное исследование нового отображения
draw(data[11])
draw(data[7])
draw(data[10])

NOUN,ADP,NOUN,NOUN,NOUN,NOUN,VERB,ADV,VERB,ADP,DET,ADJ,NOUN,.,implementation,of,georgia's,automobile,title,law,was,also,recommended,by,the,outgoing,jury,.


NOUN,VERB,merger,proposed


PRON,VERB,ADP,DET,NOUN,.,VERB,NOUN,PRT,VERB,.,DET,NOUN,.,it,urged,that,the,city,``,take,steps,to,remedy,'',this,problem,.


## Построение словарей

Для приведения коллекции документов в вид, пригодный для отправки в нейронную сеть необходимо отобразить текстовые токены в целочисленное пространство. В данном случае модель бует оперировать на уровне слов, обрабатывая одно слово за один рекуррентный шаг. Соответственно, это приведет к словарю значительного размера.

Учитывая, что словарь данного размера будет являться входящими данными для нашей нейронной сети, используя слой Embedding, можно довольно просто превратить эти данные в плотные векторы.

In [7]:
# Импортируем счетчик
from collections import Counter
# Инициализируем счет слов в наборе данных
word_counts = Counter()

# Посчитаем, сколько каждое слово встречается в наборе данных
# Результат - объект Counter, в котором содержится словар вида {слово: количество раз в наборе данных}
for sentence in data:
    words, tags = zip(*sentence)
    word_counts.update(words)

# Создаем список, содержащий элемент, соответствующие словам, не содержащимся в словаре (#UNK#), а также padding-элемент (#EOS#)
# Оставляем в этом списке только 10000 самых часто попадающихся в наборе данных слов
all_words = ['#EOS#','#UNK#'] + list(list(zip(*word_counts.most_common(10000)))[0])

# Посчитаем, какое количество слов в наборе данных покрывается словарем, созданным выше
print("Покрытие = %.5f" % (float(sum(word_counts[w] for w in all_words)) / sum(word_counts.values())))

Покрытие = 0.92876


In [8]:
# Импортируем модуль для создания словарей. От классических словарей отличается тем, что в нем невозможно возникновение KeyError
# Вместо этого на место несуществующего ключа вставляется ключ по умолчанию
from collections import defaultdict
# Создаем ивертированный индекс для словаря из прошлого шага
word_to_id = defaultdict(lambda:1, { word: i for i, word in enumerate(all_words) })
# И такой же индекс для списка меток
tag_to_id = { tag: i for i, tag in enumerate(all_tags)}

Преобразуем токены и метки в матрицу фиксированного размера

In [9]:
def to_matrix(lines, token_to_id, max_len=None, pad=0, dtype='int32', time_major=False):
    """Ковертирует список токенов в матрицу фиксированной размерности, подходящую для нейронной сети."""
    
    # Из поступающих данных выбирается строка максимальной длины или же длины, поданной как параметр
    max_len = max_len or max(map(len,lines))
    # Создается пустая матрица размерности: количество строк*маскимальная длина
    matrix = np.empty([len(lines), max_len],dtype)
    # Матрица заполняется значениями, подаными как параметр функции
    matrix.fill(pad)

    # На месте предыдущих значений записываются индексы, соответствующими словам в строках
    for i in range(len(lines)):
        line_ix = list(map(token_to_id.__getitem__,lines[i]))[:max_len]
        matrix[i,:len(line_ix)] = line_ix

    return matrix.T if time_major else matrix



In [10]:
# Проверим, что у нас получилось на трёх последних элементах набора данных (токены и метки отдельно)
batch_words, batch_tags = zip(*[zip(*sentence) for sentence in data[-3:]])

print("Матрица индексов токенов:")
print(to_matrix(batch_words, word_to_id))
print("Матрица индексов меток:")
print(to_matrix(batch_tags, tag_to_id))

Матрица индексов токенов:
[[   2 3057    5    2 2238 1334 4238 2454    3    6   19   26 1070   69
     8 2088    6    3    1    3  266   65  342    2    1    3    2  315
     1    9   87  216 3322   69 1558    4    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]
 [  45   12    8  511 8419    6   60 3246   39    2    1    1    3    2
   845    1    3    1    3   10 9910    2    1 3470    9   43    1    1
     3    6    2 1046  385   73 4562    3    9    2    1    1 3250    3
    12   10    2  861 5240   12    8 8936  121    1    4]
 [  33   64   26   12  445    7 7346    9    8 3337    3    1 2811    3
     2  463  572    2    1    1 1649   12    1    4    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]
Матрица индексов меток:
[[ 6  3  4  6  3  3  9  9  7 12  4  5  9  4  6  3 12  7  9  7  9  8  4  6
   3  7  6 13  3  4  6  3  9  4  3  7  0  0  0  0  0 

## Создадим модель

По умолчанию рекуррентный слой keras применяется ко всей последовательности входящих данных и выдает либо последовательность скрытых состояний, либо последнее скрытое состояние (в зависимости от настройки слоя). Сам рекуррентный процесс происходит автономно, "под капотом".

Завершающим слоем нашей модели должен быть полносвязный слой, применяемый на каждом временном шаге независимо от прочих шагов. Если использовать полносвязный слой Dense как есть, то он применится ко всем временным шагам в совокупности. Необходимо использовать слой TimeDistributed с целью корректной реализации как по партии, так и по времени.

In [11]:
# Импортируем библиотеку keras
import keras
# Импортируем слои
import keras.layers as L

# Инициализируем модель, в которой все слои будут располагаться последовательно
model = keras.models.Sequential()
# Слой для входящих данных
model.add(L.InputLayer([None], dtype='int32'))
# Слой для превращения целочисленных входящих матриц в плотный вектор
model.add(L.Embedding(len(all_words),50))
# Рекуррентный слой
model.add(L.SimpleRNN(64, return_sequences=True))

#add top layer that predicts tag probabilities
# Слой для предсказания вероятностей той или иной метки из списка доступных меток
stepwise_dense = L.Dense(len(all_tags), activation='softmax')
# Слой для предсказания вероятностей с учетом временных шагов, добавляем в модель именно его
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)

In [12]:
# Собираем модель
model.compile('adam','categorical_crossentropy')

In [13]:
# Смотрим на архитектуру модели
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          500100    
                                                                 
 simple_rnn (SimpleRNN)      (None, None, 64)          7360      
                                                                 
 time_distributed (TimeDistr  (None, None, 14)         910       
 ibuted)                                                         
                                                                 
Total params: 508,370
Trainable params: 508,370
Non-trainable params: 0
_________________________________________________________________


## Обучение модели

В данном случае, учитывая, что размерность матриц в каждой партии будет зависеть от максимальной длины предложения в этой партии, целесообразно использование генераторов и соответствующего метода для обучения модели.

In [14]:
# Создаем генератор для подачи входящих данных в обучение партиями
# Обработка данных генераторами позволяет использовать и ЦПУ для генерирования партий, и ГПУ для обучения
# максимальным образом используя возможности системы
from keras.utils.np_utils import to_categorical
BATCH_SIZE=32

# Функция-генератор
def generate_batches(sentences,batch_size=BATCH_SIZE,max_len=None,pad=0):
    assert isinstance(sentences,np.ndarray),"Make sure sentences is q numpy array"
    
    while True:
        indices = np.random.permutation(np.arange(len(sentences)))
        for start in range(0,len(indices)-1,batch_size):
            batch_indices = indices[start:start+batch_size]
            batch_words,batch_tags = [],[]
            for sent in sentences[batch_indices]:
                words,tags = zip(*sent)
                batch_words.append(words)
                batch_tags.append(tags)

            batch_words = to_matrix(batch_words,word_to_id,max_len,pad)
            batch_tags = to_matrix(batch_tags,tag_to_id,max_len,pad)

            batch_tags_1hot = to_categorical(batch_tags,len(all_tags)).reshape(batch_tags.shape+(-1,))
            yield batch_words,batch_tags_1hot
        

Еще один необходимый элемент контроля качества обучения модели - измерение её эффективности.

In [15]:
# Функция для расчета точности модели на данных для проверки
def compute_test_accuracy(model):
    test_words,test_tags = zip(*[zip(*sentence) for sentence in test_data])
    test_words,test_tags = to_matrix(test_words,word_to_id),to_matrix(test_tags,tag_to_id)

    # Предсказание вероятностей меток
    predicted_tag_probabilities = model.predict(test_words,verbose=1)
    predicted_tags = predicted_tag_probabilities.argmax(axis=-1)

    # Расчёт точности, исключая технические метки padding
    numerator = np.sum(np.logical_and((predicted_tags == test_tags),(test_words != 0)))
    denominator = np.sum(test_words != 0)
    return float(numerator)/denominator

# В конце каждой эпохи обучения, специальный случай callback
class EvaluateAccuracy(keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs=None):
        sys.stdout.flush()
        print("\nИзмеряю точность на данных для валидации...")
        acc = compute_test_accuracy(self.model)
        print("\nТочность на данных для валидации: %.5f\n"%acc)
        sys.stdout.flush()

In [16]:
# Обучение модели, 5 эпох
model.fit_generator(generate_batches(train_data), len(train_data)/BATCH_SIZE,
                   callbacks=[EvaluateAccuracy()], epochs=5)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


1344/1343 [==============================] - ETA: 0s - loss: 0.2646
Измеряю точность на данных для валидации...
448/448 [==============================] - 6s 13ms/step

Точность на данных для валидации: 0.93993

1343/1343 [==============================] - 78s 56ms/step - loss: 0.2646
Epoch 2/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0595
Измеряю точность на данных для валидации...
448/448 [==============================] - 6s 14ms/step

Точность на данных для валидации: 0.94425

1343/1343 [==============================] - 77s 58ms/step - loss: 0.0595
Epoch 3/5
1343/1343 [============================>.] - ETA: 0s - loss: 0.0518
Измеряю точность на данных для валидации...
448/448 [==============================] - 6s 14ms/step

Точность на данных для валидации: 0.94679

1343/1343 [==============================] - 77s 57ms/step - loss: 0.0518
Epoch 4/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0473
Измеряю точность на данных для валидации...


## Проверка точности модели

Измеряем окончательную точность на данных для проверки.

In [17]:
acc = compute_test_accuracy(model)
print("Точность на данных для проверки: %.5f"%acc)

assert acc > 0.94, "Результаты так себе"

448/448 [==============================] - 6s 13ms/step
Точность на данных для проверки: 0.94524


# Двунаправленные рекуррентные нейронные сети

Учитывая, что мы анализируем за один подход целое предложение, совершенно нормально для предсказания опираться не только на предыдущие данные, но и на последующие (согласно специфике естественных языков). Самый простой способ реализовать этот подход заключается в использовании двунаправленных рекуррентных нейронных сетей.

In [18]:
# Создаем архитектуру модели, использующей двунаправленный рекуррентный слой
model = keras.models.Sequential()

# Слой для входящих данных
model.add(L.InputLayer([None], dtype='int32'))
# Слой для превращения целочисленных входящих матриц в плотный вектор
model.add(L.Embedding(len(all_words), 50))
# Рекуррентный двунаправленный слой, реализующий слой long short term memory
model.add(L.Bidirectional(L.LSTM(64, return_sequences=True)))
# Слой для предсказания вероятностей той или иной метки из списка доступных меток
stepwise_dense = L.Dense(len(all_tags), activation='softmax')
# Слой для предсказания вероятностей с учетом временных шагов, добавляем в модель именно его
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)


In [19]:
# Собираем модель
model.compile('adam','categorical_crossentropy')

In [20]:
# Визуализируем архитектуру модели
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 50)          500100    
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        58880     
 l)                                                              
                                                                 
 time_distributed_1 (TimeDis  (None, None, 14)         1806      
 tributed)                                                       
                                                                 
Total params: 560,786
Trainable params: 560,786
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Обучаем новую модель
model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


1340/1343 [============================>.] - ETA: 0s - loss: 0.2584
Измеряю точность на данных для валидации...
448/448 [==============================] - 3s 6ms/step

Точность на данных для валидации: 0.95536

1343/1343 [==============================] - 24s 14ms/step - loss: 0.2578
Epoch 2/5
1339/1343 [============================>.] - ETA: 0s - loss: 0.0442
Измеряю точность на данных для валидации...
448/448 [==============================] - 3s 6ms/step

Точность на данных для валидации: 0.96093

1343/1343 [==============================] - 18s 14ms/step - loss: 0.0442
Epoch 3/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0367
Измеряю точность на данных для валидации...
448/448 [==============================] - 3s 6ms/step

Точность на данных для валидации: 0.96416

1343/1343 [==============================] - 15s 11ms/step - loss: 0.0367
Epoch 4/5
1342/1343 [============================>.] - ETA: 0s - loss: 0.0318
Измеряю точность на данных для валидации...
448

In [22]:
# Рассчитываю точность на данных для проверки
acc = compute_test_accuracy(model)
print("\nОкончательная точность на тестовых данных: %.5f"%acc)

assert acc > 0.96, "Должно было быть лучше"
print("Отличная работа!")

448/448 [==============================] - 3s 6ms/step

Окончательная точность на тестовых данных: 0.96523
Отличная работа!
